$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import os
import numpy as np
import subprocess
import multiprocessing as mp
import shutil

n_cores = mp.cpu_count()
MG5_Path = os.path.join(os.sep, 'Collider', 'MG5_aMC_v3_1_0', 'bin', 'mg5_aMC')
MG5_Path = os.path.relpath(MG5_Path, os.getcwd())
nevts = 500
mass_step = 500 # GeV
g_u_step = 1/4 # 
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.arange(0.5, 3.5 + g_u_step, g_u_step)


UFO_name = "Mod2_VLQ_UFO"
case = "woRHC"

header = f"""
set zerowidth_tchannel False
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > zp > ta ta @0 QED = 0 QCD = 0 
"""
proc_file_name = f'proc_zp_tau_tau_{case}.mg5'
file = open(proc_file_name,'w')
file.write(header)

# Ensure that tmp folder is empty
shutil.rmtree('tmp', ignore_errors=True)
os.makedirs('tmp', exist_ok=True)

output_folders = {}
for m in M_U:
    output_folders[m] = {}
    for g in g_U:
        output_folder = "tmp/M{}_gU{:.4f}".format(m, g).replace('.', '_')
        file.write(f"output {output_folder} -nojpeg\n")
        output_folders[m][g] = output_folder
file.close()

In [2]:
try :
    subprocess.run([MG5_Path, proc_file_name], stdout=subprocess.DEVNULL, check=True)
except :
    print("Madgraph is not installed in {}".format(MG5_Path))

In [3]:
# generate random seeds
def semilla():
    seed = np.random.randint(1,5000)
    if (seed in seeds):
        return semilla()
    return seed

# used random seeds
seeds = []

for m in M_U:
    for g in g_U:
        # create generate_events.mg5
        output_folder_abs = os.path.join( os.getcwd(), output_folders[m][g])
        # ensures that the output folder is created
        os.makedirs(output_folder_abs, exist_ok=True)
        
        with open(os.path.join(output_folder_abs, 'generate_events.mg5'), 'w') as file:
            label = f"M{m}_gU{g:.4f}".replace('.', '_')
            file.write(f"launch {output_folder_abs} -m \n")
            file.write(f"{n_cores}\ndone\n")
            # generate random seeds
            seed = semilla()
            seeds.append(seed)
            file.write(f"set iseed {seed}\n")
            # set number of events
            file.write(f"set nevents {nevts}\n")
            # set generation cuts
            file.write("set cut_decays True\n")
            file.write("set ptb 30\n")
            file.write("set ptj 20\n")
            file.write("set ptl 20\n")
            file.write("set etab 2.5\n")
            file.write("set pt_min_pdg {15: 30}\n")
            file.write("set eta_max_pdg {15: 2.5}\n")
            file.write("set mmll 20\n")
            # set paramcard
            param_card_file = os.path.join(os.getcwd(), f"Paramcards_{case}", f"param_card_{label}_dat")
            # ensures that the paramcard exists
            if not os.path.isfile(param_card_file):
                print(f"Paramcard {param_card_file} does not exist")
                raise Exception(f"Paramcard {param_card_file} does not exist")
            file.write(f"{param_card_file}\n")
            # done
            file.write("done\n")

        # launch generate_events.mg5
        try :
            subprocess.run([MG5_Path, os.path.join(output_folder_abs, 'generate_events.mg5')], stdout=subprocess.DEVNULL, check=True)
            # save LHE event files 
            source = os.path.join(output_folder_abs, 'Events', 'run_01', 'unweighted_events.lhe.gz')
            target = os.path.join("zp_tau_tau_13TeV", "Events", f"{label}", 'unweighted_events.lhe.gz')
            os.makedirs(os.path.dirname(target), exist_ok=True)
            os.rename(source, target)

            # save the cross section html file
            source = os.path.join(output_folder_abs, 'crossx.html')
            target = os.path.join("zp_tau_tau_13TeV", "Cross_Section", f'crossx_{label}.html')
            os.makedirs(os.path.dirname(target), exist_ok=True)
            os.rename(source, target)

        except :
            print("Madgraph is not installed in {}".format(MG5_Path))
# clean tmp folder
subprocess.run(["rm", "-rf", "tmp"])

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device
stty: 'sta

CompletedProcess(args=['rm', '-rf', 'tmp'], returncode=0)

In [5]:
import pandas as pd
import numpy as np
import os 
mass_step = 500 # GeV
g_u_step = 1/4 # 
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.arange(0.5, 3.5 + g_u_step, g_u_step)

xs = np.zeros((len(g_U), len(M_U)))
for j in range(len(M_U)):
    for i in range(len(g_U)):
        m=int(M_U[j])
        g=float(g_U[i])
        etiqueta = f"M{m}_gU{g:.4f}".replace('.', '_')
        ruta_tabla = f"zp_tau_tau_13TeV/Cross_Section/crossx_{etiqueta}.html"
        tabla_ij = pd.read_html(ruta_tabla)
        columna_xs = tabla_ij[0]['Cross section (pb)']
        xs[i,j] = float(columna_xs[0].split(' ')[0])         

XS_Matriz = pd.DataFrame(xs)
XS_Matriz.columns = M_U
XS_Matriz.index = g_U

# save excel file
XS_Matriz.to_excel(os.path.join("zp_tau_tau_13TeV", "Cross_Section", "Cross_Section.xlsx"))

# save csv file
XS_Matriz.to_csv(os.path.join("zp_tau_tau_13TeV", "Cross_Section", "Cross_Section.csv"))


In [9]:
!pip install openpyxl

     |████████████████████████████████| 249 kB 16.6 MB/s eta 0:00:01
